## Simple optimization model

In [ ]:
import os
import numpy as np
from gurobipy import Model, GRB, quicksum
from Input_generator import generate_input

# ---------- user knobs ----------
N_list = list(range(100, 1001, 100))   # sample sizes to run
OUT_OF_SAMPLE_M = 2000                 # fresh scenarios for OOS evaluation
OUTPUT_DIR = "results_eev_ws"          # output folder for CSVs
SEED_OOS = 12345
# ---------------------------------


In [ ]:

def build_model(J, T, l_max, l_min, l0, pi_max, pi_min, R, prices_t, inflows_tj, g):
    """Single-scenario model (hard bounds, no penalties)."""
    model = Model("hydro_revenue_max")
    model.Params.OutputFlag = 0
    Jset, Tset = range(J), range(T)

    # decision vars
    pi = model.addVars(((j, t) for j in Jset for t in Tset),
                       lb={(j, t): float(pi_min[j]) for j in Jset for t in Tset},
                       ub={(j, t): float(pi_max[j]) for j in Jset for t in Tset},
                       name="pi")

    l  = model.addVars(((j, t) for j in Jset for t in range(T+1)),
                       lb={(j, t): float(l_min[j]) for j in Jset for t in range(T+1)},
                       ub={(j, t): float(l_max[j]) for j in Jset for t in range(T+1)},
                       name="l")

    # free spill (no penalty)
    s  = model.addVars(((j, t) for j in Jset for t in Tset), lb=0.0, name="spill")

    # initial levels
    for j in Jset:
        model.addConstr(l[j, 0] == float(l0[j]))

    # dynamics: l_{j,t+1} = l_{j,t} + inflow_{t,j} + sum_k R[j,k]*pi_{k,t} - s_{j,t}
    for t in Tset:
        for j in Jset:
            model.addConstr(
                l[j, t+1] == l[j, t]
                + float(inflows_tj[t, j])
                + quicksum(float(R[j, k]) * pi[k, t] for k in Jset)
                - s[j, t]
            )

    # objective: revenue only
    revenue = quicksum(
        float(prices_t[t]) * quicksum(float(g[j]) * pi[j, t] for j in Jset)
        for t in Tset
    )
    model.setObjective(revenue, GRB.MAXIMIZE)
    return model, pi, l, s

def solve_ws_for_scenario(params, prices_t, inflows_tj):
    """Wait-and-See (perfect info) for one scenario. Returns (obj, schedule) or (nan, None)."""
    J, T, l_max, l_min, l0, pi_max, pi_min, R, g = params
    model, pi, l, s = build_model(J, T, l_max, l_min, l0, pi_max, pi_min, R, prices_t, inflows_tj, g)
    model.optimize()
    if model.Status != GRB.OPTIMAL:
        return np.nan, None
    schedule = np.array([[pi[j, t].X for t in range(T)] for j in range(J)])
    return model.ObjVal, schedule

def evaluate_fixed_schedule(params, prices_t, inflows_tj, schedule):
    """
    Evaluate a fixed π schedule under hard level bounds, with pi <= schedule
    (can reduce discharge to stay feasible). Returns objective or np.nan.
    """
    J, T, l_max, l_min, l0, pi_max, pi_min, R, g = params
    model, pi, l, s = build_model(J, T, l_max, l_min, l0, pi_max, pi_min, R, prices_t, inflows_tj, g)
    for j in range(J):
        for t in range(T):
            model.addConstr(pi[j, t] <= float(schedule[j, t]))
    model.optimize()
    if model.Status != GRB.OPTIMAL:
        return np.nan
    return model.ObjVal

def _avg_clean(x):
    x = np.asarray(x, dtype=float)
    if x.size == 0:
        return np.nan
    # ignore NaNs when averaging
    return float(np.nanmean(x)) if np.any(~np.isnan(x)) else np.nan

def run_for_N(N):
    # ---- Load N scenarios ----
    (J, T, l_max, l_min, l0, pi_max, pi_min,
     price_samples, inflow_samples,
     _nu0, _rho0, R, _a_t, _b_t, _l_bar,
     alpha_energy) = generate_input(N)

    # Pack parameters
    params = (J, T, np.array(l_max), np.array(l_min), np.array(l0),
              np.array(pi_max), np.array(pi_min), np.array(R), np.array(alpha_energy))

    # ---- TRAINING SPLIT: 80% of N for EEV training ----
    train_n = max(1, int(0.8 * N))
    prices_train  = price_samples[:train_n]
    inflows_train = inflow_samples[:train_n]

    # ---- EEV schedule from training means ----
    prices_mean  = np.mean(prices_train, axis=0)   # (T,)
    inflows_mean = np.mean(inflows_train, axis=0)  # (T, J)
    _, eev_schedule = solve_ws_for_scenario(params, prices_mean, inflows_mean)

    # ---- In-sample evaluation (across all N) ----
    eev_is, ws_is = [], []
    for s in range(N):
        ws_obj, _ = solve_ws_for_scenario(params, price_samples[s], inflow_samples[s])
        ws_is.append(ws_obj)
        eev_obj   = evaluate_fixed_schedule(params, price_samples[s], inflow_samples[s], eev_schedule)
        eev_is.append(eev_obj)

    # ---- Out-of-sample scenarios ----
    np.random.seed(SEED_OOS)
    (J_o, T_o, l_max_o, l_min_o, l0_o, pi_max_o, pi_min_o,
     price_oos, inflow_oos,
     _nu0_o, _rho0_o, R_o, _a_t_o, _b_t_o, _l_bar_o,
     alpha_energy_o) = generate_input(OUT_OF_SAMPLE_M)

    params_o = params  # same structure

    eev_oos, ws_oos = [], []
    for s in range(OUT_OF_SAMPLE_M):
        ws_obj, _ = solve_ws_for_scenario(params_o, price_oos[s], inflow_oos[s])
        ws_oos.append(ws_obj)
        eev_obj   = evaluate_fixed_schedule(params_o, price_oos[s], inflow_oos[s], eev_schedule)
        eev_oos.append(eev_obj)

    # ---- Write per-scenario CSVs ----
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    with open(os.path.join(OUTPUT_DIR, f"in_sample_N{N}.csv"), "w") as f:
        f.write("scenario,WS,EEV,regret\n")
        for idx, (w, e) in enumerate(zip(ws_is, eev_is), 1):
            wv = "" if np.isnan(w) else f"{w:.8f}"
            ev = "" if np.isnan(e) else f"{e:.8f}"
            rv = "" if (np.isnan(w) or np.isnan(e)) else f"{(w - e):.8f}"
            f.write(f"{idx},{wv},{ev},{rv}\n")

    with open(os.path.join(OUTPUT_DIR, f"out_of_sample_N{N}.csv"), "w") as f:
        f.write("scenario,WS,EEV,regret\n")
        for idx, (w, e) in enumerate(zip(ws_oos, eev_oos), 1):
            wv = "" if np.isnan(w) else f"{w:.8f}"
            ev = "" if np.isnan(e) else f"{e:.8f}"
            rv = "" if (np.isnan(w) or np.isnan(e)) else f"{(w - e):.8f}"
            f.write(f"{idx},{wv},{ev},{rv}\n")

    # EEV schedule (rows=time, cols=reservoirs)
    with open(os.path.join(OUTPUT_DIR, f"eev_schedule_N{N}.csv"), "w") as f:
        header = ",".join([f"Reservoir{j+1}" for j in range(J)])
        f.write("time," + header + "\n")
        for t in range(T):
            row = ",".join(f"{eev_schedule[j, t]:.8f}" for j in range(J))
            f.write(f"{t+1},{row}\n")

    # ---- Return summary row ----
    return {
        "N": N,
        "EEV_in_sample": _avg_clean(eev_is),
        "WS_in_sample":  _avg_clean(ws_is),
        "EEV_out_of_sample": _avg_clean(eev_oos),
        "WS_out_of_sample":  _avg_clean(ws_oos),
    }

def main():
    rows = []
    for N in N_list:
        rows.append(run_for_N(N))

    # write one summary CSV with averages per N
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    summary_path = os.path.join(OUTPUT_DIR, "summary.csv")
    with open(summary_path, "w") as f:
        f.write("N,EEV_in_sample,WS_in_sample,EEV_out_of_sample,WS_out_of_sample\n")
        for r in rows:
            def fmt(x): 
                return "" if (x is None or (isinstance(x, float) and np.isnan(x))) else f"{x:.8f}"
            f.write(f"{r['N']},{fmt(r['EEV_in_sample'])},{fmt(r['WS_in_sample'])},"
                    f"{fmt(r['EEV_out_of_sample'])},{fmt(r['WS_out_of_sample'])}\n")

if __name__ == "__main__":
    main()
